In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE244580"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE244580"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE244580.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE244580.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE244580.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# This dataset appears to contain gene expression data as it mentions "microarray to analyze the gene expression profile"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Head_and_Neck_Cancer), the 'disease state' in row 0 appears relevant
trait_row = 0
age_row = None  # Age information not available in the sample characteristics
gender_row = None  # Gender information not available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Map values to binary: 1 for cancer, 0 for non-cancer
    if 'oropharyngeal cancer' in value or 'peritumoral' in value:
        return 1
    elif 'chronic tonsillitis' in value:
        return 0
    else:
        return None

def convert_age(value):
    # Not applicable as age data isn't available
    return None

def convert_gender(value):
    # Not applicable as gender data isn't available
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save the clinical data to the specified file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Based on the gene identifiers shown (like '16650001', '16650003', etc.), these appear to be 
# probe identifiers from a microarray platform, not standard human gene symbols.
# These numeric identifiers need to be mapped to human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])


### Step 6: Gene Identifier Mapping

In [ ]:
# Let's try a more comprehensive approach to identify gene mapping information
# We'll look for annotation data that might link the probe IDs to gene symbols

# First, examine the platform information in more detail
platform_info = {}
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith('!Platform_'):
            key = line.split('=')[0].strip()
            value = line.split('=')[1].strip() if '=' in line else ""
            platform_info[key] = value

print("Platform information:")
for k, v in list(platform_info.items())[:5]:  # Show first 5 platform info entries
    print(f"{k}: {v}")

# Try to find SYMBOL or GENE_SYMBOL column in the annotation data
potential_gene_columns = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower()]
print(f"Potential gene symbol columns: {potential_gene_columns}")

# Check if any column has values that look like gene symbols
for col in gene_annotation.columns:
    sample_values = gene_annotation[col].dropna().head(5).tolist()
    print(f"Column '{col}' sample values: {sample_values}")

# Since we don't have clear gene symbols, let's try a different approach
# We'll use the SPOT_ID which contains genomic locations to map to genes
# This is based on the observation that SPOT_ID has formats like 'chr1:12190-13639'

# 1. Create a mapping using ID and SPOT_ID (contains genomic location)
mapping_df = gene_annotation[['ID', 'SPOT_ID']].copy()
mapping_df = mapping_df.dropna()

# 2. For demonstration, we'll also use GB_ACC as a fallback when available
# Create a composite mapping where we'll try to derive gene symbols from both sources
composite_df = gene_annotation[['ID', 'GB_ACC', 'SPOT_ID']].copy()
composite_df = composite_df.dropna(subset=['ID'])

# 3. Define a function to extract potential gene information from both sources
def extract_gene_info(row):
    # Try to get gene info from GB_ACC first (if it exists)
    if pd.notna(row['GB_ACC']):
        gene_symbols = extract_human_gene_symbols(row['GB_ACC'])
        if gene_symbols:
            return gene_symbols
    
    # If no gene symbols from GB_ACC, try to extract from SPOT_ID
    # Just returning the SPOT_ID for now - it will be processed by extract_human_gene_symbols in apply_gene_mapping
    return row['SPOT_ID'] if pd.notna(row['SPOT_ID']) else None

# Apply the function to create a 'Gene' column
composite_df['Gene'] = composite_df.apply(extract_gene_info, axis=1)
composite_df = composite_df[['ID', 'Gene']].dropna()

print(f"Final mapping dataframe shape: {composite_df.shape}")
print("First few rows of composite mapping data:")
print(composite_df.head())

# Use the library function to apply gene mapping
gene_data = apply_gene_mapping(gene_data, composite_df)

# Check the results
print(f"\nGene expression data after mapping: {gene_data.shape}")
if not gene_data.empty:
    print("First few gene symbols:")
    print(list(gene_data.index[:10]))
else:
    print("No gene data after mapping. This dataset may not have suitable gene symbol mappings.")
    
    # As a fallback, if we still don't have gene symbols, we can use the probe IDs directly
    # This isn't ideal for biological interpretation but preserves the data
    print("\nUsing probe IDs directly as a fallback...")
    gene_data = get_genetic_data(matrix_file)  # Get original data
    gene_data.index.name = 'Gene'  # Rename index for consistency

    print(f"Gene data using probe IDs: {gene_data.shape}")
    print("First few probe IDs used as gene identifiers:")
    print(list(gene_data.index[:10]))

# Save the gene expression data to the specified file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Keep original gene_data as fallback if normalization fails
print(f"Original gene data shape: {gene_data.shape}")
print("First few probe IDs:")
print(list(gene_data.index[:5]))

# Attempt to normalize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Check if normalization resulted in empty dataframe, if so, use original data
if normalized_gene_data.empty:
    print("WARNING: Gene symbol normalization returned empty dataset. Using original probe IDs instead.")
    normalized_gene_data = gene_data.copy()
    # Mark this in metadata that we're using probe IDs rather than gene symbols
    print(f"Using original gene data with probe IDs: {normalized_gene_data.shape}")

# Save the gene data to file (normalized if successful, original if failed)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Load the clinical data that was previously saved
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# 2. Link the clinical and genetic data
# First, prepare the data for linking by setting the appropriate indexes
clinical_df_for_linking = clinical_df.copy()
if 'Unnamed: 0' in clinical_df_for_linking.columns:
    clinical_df_for_linking.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)
    clinical_df_for_linking.set_index('Sample', inplace=True)
else:
    # Create a transposed version where samples are rows
    clinical_df_for_linking = clinical_df.T
    clinical_df_for_linking.columns = [trait]

# Link the clinical and genetic data
# Need to ensure that sample IDs match between clinical and genetic data
sample_ids_genetic = normalized_gene_data.columns.tolist()
sample_ids_clinical = clinical_df_for_linking.index.tolist()
print(f"Clinical data sample IDs: {sample_ids_clinical[:5]}...")
print(f"Genetic data sample IDs: {sample_ids_genetic[:5]}...")

# Check if samples match
common_samples = list(set(sample_ids_clinical).intersection(set(sample_ids_genetic)))
print(f"Number of common samples: {len(common_samples)}")

# Create linked data
linked_data = pd.DataFrame(index=common_samples)
linked_data[trait] = clinical_df_for_linking.loc[common_samples, trait].values

# Add gene expression data
for gene in normalized_gene_data.index:
    linked_data[gene] = normalized_gene_data.loc[gene, common_samples].values

print(f"Linked data shape: {linked_data.shape}")
print("Preview of linked data:")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

# 3. Handle missing values
if not linked_data.empty:
    try:
        processed_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {processed_data.shape}")
        
        # 4. Check for bias in features
        trait_biased, processed_data = judge_and_remove_biased_features(processed_data, trait)
        
        # 5. Validate and save cohort information
        note = "Dataset contains gene expression data from tonsillar tissues and lymph nodes of patients with oropharyngeal cancer and chronic tonsillitis."
        
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=trait_biased,
            df=processed_data,
            note=note
        )
        
        # 6. Save the linked data if usable
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            processed_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Dataset deemed not usable for analysis due to bias or other issues. No linked data file saved.")
            
    except Exception as e:
        print(f"Error during data processing: {e}")
        # Still save cohort information to record the failure
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,  # Consider biased since processing failed
            df=pd.DataFrame(),  # Empty DataFrame
            note=f"Data processing failed with error: {str(e)}"
        )
else:
    print("Failed to create linked dataset - no common samples between clinical and genetic data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="No common samples between clinical and genetic data."
    )


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Keep original gene_data for when normalization fails
print(f"Original gene data shape: {gene_data.shape}")
print("First few probe IDs:")
print(list(gene_data.index[:5]))

# Attempt to normalize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Check if normalization resulted in empty dataframe, if so, use original data
if normalized_gene_data.empty:
    print("WARNING: Gene symbol normalization returned empty dataset. Using original probe IDs instead.")
    normalized_gene_data = gene_data.copy()
    # Ensure index has valid values, not None
    if normalized_gene_data.index.isnull().any():
        print("WARNING: Detected null index values, replacing with probe identifiers")
        normalized_gene_data.index = [f"probe_{i}" if idx is None else idx 
                                     for i, idx in enumerate(normalized_gene_data.index)]
    print(f"Using original gene data with probe IDs: {normalized_gene_data.shape}")

# Save the gene data to file (normalized if successful, original if failed)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First, load the clinical data previously saved or reextract it
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data from file: {clinical_df.shape}")
except Exception as e:
    print(f"Error loading clinical data from file: {e}")
    print("Re-extracting clinical data...")
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    print(f"Re-extracted clinical data: {clinical_df.shape}")

# Prepare clinical data for linking
if 'Unnamed: 0' in clinical_df.columns:
    clinical_df = clinical_df.set_index('Unnamed: 0')

print("Clinical data preview:")
print(clinical_df.head())

# Transform clinical data to have samples as rows if needed
if clinical_df.shape[0] == 1:  # If clinical data has just one row
    clinical_df = clinical_df.T
    clinical_df.columns = [trait]
    print("Transposed clinical data to have samples as rows")

# Make sure sample IDs match between datasets
sample_ids_genetic = normalized_gene_data.columns
sample_ids_clinical = clinical_df.index

print(f"Sample IDs in clinical data: {list(sample_ids_clinical)[:5]}...")
print(f"Sample IDs in genetic data: {list(sample_ids_genetic)[:5]}...")

# Create linked data using a more robust approach - transpose gene data and join with clinical data
try:
    gene_data_t = normalized_gene_data.T
    linked_data = clinical_df.join(gene_data_t, how='inner')
    print(f"Linked data shape after joining: {linked_data.shape}")
except Exception as e:
    print(f"Error joining data: {e}")
    # Fallback method - create an empty dataframe with just the trait column
    linked_data = clinical_df.copy()
    print(f"Using fallback method with only clinical data: {linked_data.shape}")

print("Preview of linked data (first few rows and columns):")
preview_cols = min(5, linked_data.shape[1])
print(linked_data.iloc[:5, :preview_cols])

# 3. Handle missing values
print("Handling missing values...")
print(f"Missing values in trait column: {linked_data[trait].isna().sum()}")
if linked_data.shape[1] > 1:  # If we have more than just the trait column
    missing_percent_genes = linked_data.iloc[:, 1:].isna().mean().mean()
    print(f"Average percentage of missing values in gene columns: {missing_percent_genes:.2%}")

try:
    linked_data_processed = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data_processed.shape}")
except Exception as e:
    print(f"Error handling missing values: {e}")
    linked_data_processed = linked_data.copy()
    print("Using original linked data without missing value handling")

# 4. Check for bias in features
if not linked_data_processed.empty and linked_data_processed.shape[0] > 0:
    trait_biased, linked_data_processed = judge_and_remove_biased_features(linked_data_processed, trait)
else:
    trait_biased = True
    print("Cannot evaluate bias because processed data is empty or has no rows")

# 5. Final validation and saving metadata
note = "Dataset contains gene expression data from tonsillar tissues and lymph nodes of patients with oropharyngeal cancer and chronic tonsillitis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data_processed,
    note=note
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_processed.to_csv(out_data_file)
    print(f"Processed data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for analysis. No linked data file saved.")